In [1]:
import numpy as np
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
from itertools import permutations 
from generator import next_graph
import timeit

Loading: e5x_1


In [2]:
def create_model(model_name, G, tree, first_node_idx = 0):
    with gp.Env(empty = True) as env:
        env.setParam('LogToConsole', 0)
        env.start()
        
        model=gp.Model(model_name, env=env)
        
        n = len(G)
        n_list = list(G)
        n_dict = {n_list[idx]: idx for idx in range(n)}
        first_node = n_list[first_node_idx]

        tree_closure = nx.transitive_closure_dag(tree)
        A = nx.to_numpy_matrix(G, nodelist=list(G))

        x_ij, cost = gp.multidict(dict(np.ndenumerate(A)))
        y_ij, dummy_cost = gp.multidict(dict(np.ndenumerate(np.zeros((n,n)))))

        ### VARIABLES
        x = model.addVars(x_ij, vtype=GRB.BINARY, name='x')
        y = model.addVars(y_ij, name='y')

        ### OBJECTIVE
        objective = model.setObjective(x.prod(cost), GRB.MINIMIZE)

        ### CONSTRAINTS
        ### FLOW CONSERVATION
        for v_i in range(n):
            model.addConstr(sum(x[v_i,v_j] for v_j in range(n) 
                                                if (n_list[v_i],n_list[v_j]) in G.edges) == 1, f'fc_outer_{n_list[v_i]}')

            model.addConstr(sum(x[v_j,v_i] for v_j in range(n) 
                                                if (n_list[v_j],n_list[v_i]) in G.edges) == 1, f'fc_inner_{n_list[v_i]}')
  
    

        ### SUB-TOUR ELIMINATION
        for e in G.edges:
            i = n_dict[e[0]]
            j = n_dict[e[1]]

            if not first_node_idx in (i,j):               
                model.addConstr(y[i,j] - x[i,j] >= 0, f'ste1_{n_list[i]}_{n_list[j]}')
                model.addConstr(y[i,j] + y[j,i] == 1, f'ste2_{n_list[i]}_{n_list[j]}')

        for i,j,k in [p for p in permutations(range(1,n),3) if sorted(list(p)) == list(p)]:
            model.addConstr(y[i,j] + y[j,k] + y[k,i] <= 2, f'ste3_{n_list[i]}_{n_list[j]}_{n_list[k]}')

        ### PRECEDENCE CONSTRAINTS

        for e in tree_closure.edges:
            i = n_dict[e[0]]
            j = n_dict[e[1]]
            
            model.addConstr(y[i,j] == 1, f'pc_{n_list[i]}_{n_list[j]}')

        return model, x


def optimizeModel(model):
    model.setParam(GRB.Param.TimeLimit,1)
    model.optimize()
    
    return model.status, model.objboundc
    
    


In [3]:
G=nx.DiGraph()
edge_list = [('1','2',10),('1','3',15),('2','3',5), ('3','4',10), ('4','2', 25), ('4','1', 35)]
G.add_weighted_edges_from(edge_list)
A=nx.to_numpy_matrix(G, nodelist=list(G))
len(G)
tree = nx.DiGraph()
tree.add_edges_from([('1','2'),('1','3')])
nd_list = list(G)
{nd_list[idx]: idx for idx in range(len(nd_list))}

{'1': 0, '2': 1, '3': 2, '4': 3}

In [4]:
G2 = nx.DiGraph()
edge_list2 = [(1,2,1), (2,3,0), (3,1,1), (2,4,1), (4,5,1), (5,6,1), (6,4,0), (6,3,1)]
G2.add_weighted_edges_from(edge_list2)
A2=nx.to_numpy_matrix(G2, nodelist=list(G2))
tree2=nx.DiGraph()
tree2.add_edges_from([(1,2), (2,4), (2,3)])

In [5]:
def to_measure():
    G3, tree3, sigma = next_graph()
    print(f'sigma={sigma}')

    model, x = create_model('Toy2', G3, tree3)
    model.write('model-toy2.lp')
    res = optimizeModel(model)
    print(f'Result: {res}')

In [6]:
%timeit to_measure()

Size:	 315 / 23
UB:	 1890.0

sigma=(1,)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 942.0)
sigma=(1, 9)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 927.0)
sigma=(1, 11)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 857.0)
sigma=(1, 13)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 901.0)
sigma=(1, 15)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 739.0)
sigma=(1, 17)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 904.0)
sigma=(1, 19)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 827.0)
sigma=(1, 21)
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Result: (2, 931.0)
71.6 ms ± 1.42 ms per loop (mean ± std.